In [1]:
#
# Vinh Phuc Ta Dang ft Dao Cong Tinh
#
import os
import cv2
import time
import posenet
import numpy as np
from constants import *
# import tensorflow as tf
from scipy.spatial.distance import euclidean
from tensorflow.compat.v1.keras import backend as K

VID_EXT_VALIDS = ['.mp4', '.mov']
scale_factor = 0.5
VIDEO_URI = 0
GUI_Enable = False
oo = 1e9

In [16]:

def normalize(keypoint_coords):
    first_keypoint = keypoint_coords[0]
    nose, feature = first_keypoint[0], []
    mn, mx = oo, -oo
    for i in range(1, len(first_keypoint)):
        dist = euclidean(nose, first_keypoint[i])
        feature.append(dist)

    feature = np.array(feature)
    mx = np.max(feature)
    mn = np.min(feature)
    # norm = np.linalg.norm(feature)
    # normalized_feature = feature/norm
    normalized_feature = np.array([(val-mn)/(mx-mn) for val in feature])
    return normalized_feature

def extractFeatures(VIDEO_URI):
    sess = K.get_session()
    model_cfg, model_outputs = posenet.load_model(101, sess)
    output_stride = model_cfg['output_stride']

    cap = cv2.VideoCapture(VIDEO_URI)
    flip = False
    cap.set(3, 257)
    cap.set(4, 257)
    frame_count = 0
    frame_series, frame_seq = [], []

    print('Extracting video ' + VIDEO_URI + '...')
    while True:
        try:
            input_image, display_image, output_scale = posenet.read_cap(
                cap, flip=flip, scale_factor=scale_factor, output_stride=output_stride)
        except:
            print("End video %s.%d\n" % (VIDEO_URI, len(frame_series)))
            break

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            min_pose_score=0.15)
            
        keypoint_coords *= output_scale
        normalized_feature = normalize(keypoint_coords)
        frame_count += 1
        frame_seq.append(normalized_feature)
        if frame_count % window_size == 0:
            frame_series.append(frame_seq)
            frame_seq = []

        # TODO this isn't particularly fast, use GL for drawing and display someday...
        if GUI_Enable:
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)

            cv2.imshow('posenet', overlay_image)
            # if "running" in VIDEO_URI:
            #     filename = "running.png"
            # else:
            #     filename = "walking.png"
            # cv2.imwrite(filename, overlay_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
    return np.array(frame_series)

In [17]:
# print(extractFeatures('pose/datasets/running/Tinh_Running_2.mov'))
# print(extractFeatures('pose/datasets/walking/Phuc_Walking.mp4'))

Extracting video pose/datasets/running/Tinh_Running_2.mov...
[[7.75770460e-03 5.15122961e-01]
 [7.73856831e-04 6.08819253e-01]
 [0.00000000e+00 2.45600252e-01]
 [2.29972798e-02 7.89687759e-01]
 [2.06174680e-02 1.58789830e-01]
 [1.78470705e-01 8.05972778e-01]
 [1.87210506e-01 5.54768019e-02]
 [3.50872104e-01 8.72332076e-01]
 [3.35904967e-01 0.00000000e+00]
 [5.22857741e-01 1.00000000e+00]
 [5.02983120e-01 1.45080366e-01]
 [5.26949199e-01 7.82479506e-01]
 [5.30622342e-01 1.04267027e-01]
 [7.77880381e-01 5.02342116e-01]
 [7.90137460e-01 2.69643387e-02]
 [1.00000000e+00 4.48695648e-01]
 [9.96116519e-01 4.67406093e-02]]
[  5.94028482  15.59840484  16.63024944  20.82827443  66.36403261
  72.4389477  130.76442503 126.9664876  195.87752801 187.62875657
 196.05862864 198.23235921 289.94168481 295.85987018 373.5848952
 373.05525888]
[]
Extracting video pose/datasets/walking/Phuc_Walking.mp4...
[[2.27050701e-02 5.15368158e-01]
 [0.00000000e+00 6.10626755e-01]
 [6.85451926e-04 4.02514893e-01]
 [2.

In [ ]:
actions = np.array([])
labels = np.array([])
datasets_dir = 'pose/datasets'

time_start = time.time()
for action in os.listdir(datasets_dir):
    action_dir = os.path.join(datasets_dir, action)
    if '.DS_Store' not in action_dir:
        print("\n############# ACTION: %s #############\n" % action)
        total = 0
        for vid in os.listdir(action_dir):
            _, ext = os.path.splitext(vid)
            if ext in VID_EXT_VALIDS:
                video_path = os.path.join(action_dir, vid)
                keyPoints = extractFeatures(video_path)
                labels = np.append(labels, np.array(
                    [action]*keyPoints.shape[0]))
                total += keyPoints.shape[0]
                if len(actions) == 0:
                    actions = keyPoints
                else:
                    actions = np.vstack((actions, keyPoints))
        print("Total: %d" % total)
time_end = time.time()

In [ ]:
# View result
print('Total-time: %.6f (s)' % (time_end - time_start))
print('actions.shape =', actions.shape)
print('labels.shape =', labels.shape)

# Store data numpy array
np.save('normalized_actions.npy', actions)
np.save('normalized_labels.npy', labels)